In [ ]:
https://www.kaggle.com/aaditkapoor1201/iris-classification-pytorch

In [139]:
# import package
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F

import matplotlib.pyplot as plt

# set numpy seed for reproducibility
np.random.seed(seed=123)

In [142]:
dataset = pd.read_csv("../data/iris.csv", sep=',', header=None)
dataset.columns = ['f1', 'f2', 'f3', 'f4', 'species']
dataset.head()

,f1,f2,f3,f4,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [143]:
#dataset["label"] = dataset["label"].astype('category')
#dataset['label_cat'] = dataset['label'].cat.codes.astype('float64')
#dataset = dataset[['f1', 'f2', 'f3', 'f4', 'label_cat']]
dataset.loc[dataset.species=='Iris-setosa', 'species'] = 0
dataset.loc[dataset.species=='Iris-versicolor', 'species'] = 1
dataset.loc[dataset.species=='Iris-virginica', 'species'] = 2

#dataset = pd.get_dummies(dataset, prefix=['label'])
dataset.head()

,f1,f2,f3,f4,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [144]:
dataset.dtypes

f1         float64
f2         float64
f3         float64
f4         float64
species      int64
dtype: object

In [145]:
dataset = dataset.values

In [146]:
train_size = 0.8
train_size = int(len(dataset) * train_size)
np.random.shuffle(dataset)
train, test = dataset[:train_size,:], dataset[train_size:,:]
print ('Dataset size: {0}. Train size: {1} - Test size: {2}'.format(len(dataset), len(train), len(test)))

Dataset size: 150. Train size: 120 - Test size: 30


In [147]:
# get train and set data into numpy nd.array
x_train = train[:,:4]
x_test = test[:,:4]
y_train = train[:,4]
y_test = test[:,4]

In [148]:
y_train

array([1., 2., 2., 1., 0., 2., 1., 0., 0., 1., 2., 0., 1., 2., 2., 2., 0.,
       0., 1., 0., 0., 2., 0., 2., 0., 0., 0., 2., 2., 0., 2., 2., 0., 0.,
       1., 1., 2., 0., 0., 1., 1., 0., 2., 2., 2., 2., 2., 1., 0., 0., 2.,
       0., 0., 1., 1., 1., 1., 2., 1., 2., 0., 2., 1., 0., 0., 2., 1., 2.,
       2., 0., 1., 1., 2., 0., 2., 1., 1., 0., 2., 2., 0., 0., 1., 1., 2.,
       0., 0., 1., 0., 1., 2., 0., 2., 0., 0., 1., 0., 0., 1., 2., 1., 1.,
       1., 0., 0., 1., 2., 0., 0., 1., 1., 1., 2., 1., 1., 1., 2., 0., 0.,
       1.])

In [149]:
batch_size = len(x_train) # 50
num_epochs = 1000
learning_rate = 0.01
size_hidden = 100
n_feature = 4
n_output = 3
batch_no = int(len(x_train) / batch_size)

In [ ]:
class Net(nn.Module):
    # define nn
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4, 100)
        self.fc2 = nn.Linear(100, 100)
        self.fc3 = nn.Linear(100, 3)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, X):
        X = F.relu(self.fc1(X))
        X = self.fc2(X)
        X = self.fc3(X)
        X = self.softmax(X)

        return X

In [152]:
net = torch.nn.Sequential(
    torch.nn.Linear(n_feature, size_hidden),
    torch.nn.ReLU(),
    torch.nn.Linear(size_hidden, size_hidden),
    torch.nn.Linear(size_hidden, n_output),
    torch.nn.Softmax(dim=1)
)

In [153]:
criterion = torch.nn.CrossEntropyLoss()

In [154]:
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

In [155]:
from sklearn.utils import shuffle
from torch.autograd import Variable
running_loss = 0.0

#inputs = Variable(torch.tensor(x_train).float())
#labels = Variable(torch.tensor(y_train).float())

batch_no

for epoch in range(num_epochs):
    
    for i in range(batch_no):
        start = i * batch_size
        end = start + batch_size
        inputs = Variable(torch.tensor(x_train[start:end]).float())
        labels = Variable(torch.tensor(y_train[start:end]).long())
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        #print("outputs",outputs)
        #print("outputs",outputs,outputs.shape,"labels",labels, labels.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        
    print('Epoch {}'.format(epoch+1), "loss: ",running_loss)
    running_loss = 0.0

Epoch 1 loss:  1.1241374015808105
Epoch 2 loss:  1.1141802072525024
Epoch 3 loss:  1.1038709878921509
Epoch 4 loss:  1.0938069820404053
Epoch 5 loss:  1.0845600366592407
Epoch 6 loss:  1.076492428779602
Epoch 7 loss:  1.069683313369751
Epoch 8 loss:  1.0639945268630981
Epoch 9 loss:  1.0591870546340942
Epoch 10 loss:  1.0550187826156616
Epoch 11 loss:  1.0512880086898804
Epoch 12 loss:  1.047844409942627
Epoch 13 loss:  1.0445818901062012
Epoch 14 loss:  1.0414268970489502
Epoch 15 loss:  1.0383312702178955
Epoch 16 loss:  1.0352617502212524
Epoch 17 loss:  1.032198429107666
Epoch 18 loss:  1.0291260480880737
Epoch 19 loss:  1.0260354280471802
Epoch 20 loss:  1.022922396659851
Epoch 21 loss:  1.0197843313217163
Epoch 22 loss:  1.016621708869934
Epoch 23 loss:  1.0134360790252686
Epoch 24 loss:  1.010230541229248
Epoch 25 loss:  1.00700843334198
Epoch 26 loss:  1.0037754774093628
Epoch 27 loss:  1.0005381107330322
Epoch 28 loss:  0.9973028898239136
Epoch 29 loss:  0.9940769076347351
Epo

Epoch 240 loss:  0.7756052613258362
Epoch 241 loss:  0.7751644849777222
Epoch 242 loss:  0.7747243046760559
Epoch 243 loss:  0.7742855548858643
Epoch 244 loss:  0.7738475203514099
Epoch 245 loss:  0.7734108567237854
Epoch 246 loss:  0.7729748487472534
Epoch 247 loss:  0.7725395560264587
Epoch 248 loss:  0.7721054553985596
Epoch 249 loss:  0.771671712398529
Epoch 250 loss:  0.7712393999099731
Epoch 251 loss:  0.7708075642585754
Epoch 252 loss:  0.7703769207000732
Epoch 253 loss:  0.7699469327926636
Epoch 254 loss:  0.7695177793502808
Epoch 255 loss:  0.7690893411636353
Epoch 256 loss:  0.7686624526977539
Epoch 257 loss:  0.7682355642318726
Epoch 258 loss:  0.7678101062774658
Epoch 259 loss:  0.7673851847648621
Epoch 260 loss:  0.7669610977172852
Epoch 261 loss:  0.7665374875068665
Epoch 262 loss:  0.7661149501800537
Epoch 263 loss:  0.7656928300857544
Epoch 264 loss:  0.7652714252471924
Epoch 265 loss:  0.764851450920105
Epoch 266 loss:  0.7644315958023071
Epoch 267 loss:  0.76401346921

Epoch 478 loss:  0.6898252367973328
Epoch 479 loss:  0.6895438432693481
Epoch 480 loss:  0.6892633438110352
Epoch 481 loss:  0.6889838576316833
Epoch 482 loss:  0.6887046694755554
Epoch 483 loss:  0.6884262561798096
Epoch 484 loss:  0.6881487965583801
Epoch 485 loss:  0.687872052192688
Epoch 486 loss:  0.6875959634780884
Epoch 487 loss:  0.6873204112052917
Epoch 488 loss:  0.6870455145835876
Epoch 489 loss:  0.6867715120315552
Epoch 490 loss:  0.6864978671073914
Epoch 491 loss:  0.6862255334854126
Epoch 492 loss:  0.685953676700592
Epoch 493 loss:  0.6856825351715088
Epoch 494 loss:  0.6854120492935181
Epoch 495 loss:  0.6851426959037781
Epoch 496 loss:  0.6848734021186829
Epoch 497 loss:  0.6846051812171936
Epoch 498 loss:  0.684337854385376
Epoch 499 loss:  0.6840712428092957
Epoch 500 loss:  0.6838054060935974
Epoch 501 loss:  0.6835401058197021
Epoch 502 loss:  0.6832753419876099
Epoch 503 loss:  0.6830112338066101
Epoch 504 loss:  0.682748556137085
Epoch 505 loss:  0.6824858188629

Epoch 715 loss:  0.6409346461296082
Epoch 716 loss:  0.6407923102378845
Epoch 717 loss:  0.6406499743461609
Epoch 718 loss:  0.6405085325241089
Epoch 719 loss:  0.6403675079345703
Epoch 720 loss:  0.6402269005775452
Epoch 721 loss:  0.6400865316390991
Epoch 722 loss:  0.6399468779563904
Epoch 723 loss:  0.639807403087616
Epoch 724 loss:  0.6396684050559998
Epoch 725 loss:  0.6395298838615417
Epoch 726 loss:  0.6393913626670837
Epoch 727 loss:  0.6392536163330078
Epoch 728 loss:  0.6391163468360901
Epoch 729 loss:  0.6389795541763306
Epoch 730 loss:  0.6388428807258606
Epoch 731 loss:  0.638706624507904
Epoch 732 loss:  0.6385711431503296
Epoch 733 loss:  0.6384355425834656
Epoch 734 loss:  0.638300895690918
Epoch 735 loss:  0.6381660103797913
Epoch 736 loss:  0.6380319595336914
Epoch 737 loss:  0.637898325920105
Epoch 738 loss:  0.6377649307250977
Epoch 739 loss:  0.6376323699951172
Epoch 740 loss:  0.6374997496604919
Epoch 741 loss:  0.637367844581604
Epoch 742 loss:  0.63723617792129

Epoch 956 loss:  0.6163633465766907
Epoch 957 loss:  0.6162920594215393
Epoch 958 loss:  0.6162206530570984
Epoch 959 loss:  0.6161495447158813
Epoch 960 loss:  0.6160786747932434
Epoch 961 loss:  0.6160078644752502
Epoch 962 loss:  0.6159374117851257
Epoch 963 loss:  0.6158668994903564
Epoch 964 loss:  0.6157967448234558
Epoch 965 loss:  0.615726888179779
Epoch 966 loss:  0.6156567335128784
Epoch 967 loss:  0.6155874133110046
Epoch 968 loss:  0.615517795085907
Epoch 969 loss:  0.615448534488678
Epoch 970 loss:  0.6153793334960938
Epoch 971 loss:  0.6153104305267334
Epoch 972 loss:  0.6152415871620178
Epoch 973 loss:  0.6151731014251709
Epoch 974 loss:  0.6151047348976135
Epoch 975 loss:  0.6150362491607666
Epoch 976 loss:  0.6149682402610779
Epoch 977 loss:  0.6149001717567444
Epoch 978 loss:  0.6148325800895691
Epoch 979 loss:  0.6147652864456177
Epoch 980 loss:  0.6146977543830872
Epoch 981 loss:  0.6146305799484253
Epoch 982 loss:  0.6145636439323425
Epoch 983 loss:  0.614496946334

In [171]:
outputs.data

tensor([[4.7551e-04, 3.9394e-01, 6.0558e-01],
        [4.1037e-05, 1.1331e-01, 8.8665e-01],
        [2.2359e-06, 1.4453e-02, 9.8554e-01],
        [1.3787e-02, 9.0762e-01, 7.8589e-02],
        [9.8614e-01, 1.3856e-02, 8.2584e-09],
        [6.2317e-04, 3.4587e-01, 6.5351e-01],
        [3.4013e-03, 8.3522e-01, 1.6138e-01],
        [9.8894e-01, 1.1056e-02, 1.2191e-08],
        [9.7691e-01, 2.3090e-02, 7.1753e-08],
        [2.4939e-03, 6.0676e-01, 3.9074e-01],
        [3.4333e-04, 3.4840e-01, 6.5126e-01],
        [9.9834e-01, 1.6561e-03, 2.0634e-11],
        [8.8133e-03, 7.6289e-01, 2.2830e-01],
        [5.0453e-05, 1.1983e-01, 8.8011e-01],
        [8.2221e-06, 7.3265e-02, 9.2673e-01],
        [1.7394e-06, 1.3420e-02, 9.8658e-01],
        [9.9654e-01, 3.4642e-03, 1.2719e-10],
        [9.9129e-01, 8.7055e-03, 7.3405e-09],
        [2.3834e-03, 7.2037e-01, 2.7724e-01],
        [9.8332e-01, 1.6681e-02, 1.8906e-08],
        [9.9391e-01, 6.0876e-03, 7.8572e-10],
        [3.9529e-04, 3.6465e-01, 6

In [169]:
labels

tensor([1, 2, 2, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 2, 2, 2, 0, 0, 1, 0, 0, 2, 0, 2,
        0, 0, 0, 2, 2, 0, 2, 2, 0, 0, 1, 1, 2, 0, 0, 1, 1, 0, 2, 2, 2, 2, 2, 1,
        0, 0, 2, 0, 0, 1, 1, 1, 1, 2, 1, 2, 0, 2, 1, 0, 0, 2, 1, 2, 2, 0, 1, 1,
        2, 0, 2, 1, 1, 0, 2, 2, 0, 0, 1, 1, 2, 0, 0, 1, 0, 1, 2, 0, 2, 0, 0, 1,
        0, 0, 1, 2, 1, 1, 1, 0, 0, 1, 2, 0, 0, 1, 1, 1, 2, 1, 1, 1, 2, 0, 0, 1])

In [167]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

predict_out = net(torch.tensor(x_test).float())
_, y_pred = torch.max(predict_out, 1)

print (accuracy_score(y_test, y_pred))

print (precision_score(y_test, y_pred, average='macro'))
print (precision_score(y_test, y_pred, average='micro'))
print (recall_score(y_test, y_pred, average='macro'))
print (recall_score(y_test, y_pred, average='micro'))

0.9333333333333333
0.9555555555555556
0.9333333333333333
0.9393939393939394
0.9333333333333333


In [173]:
predict_out

tensor([[1.5124e-05, 6.2422e-02, 9.3756e-01],
        [5.1592e-05, 1.1457e-01, 8.8538e-01],
        [3.9500e-05, 1.6542e-01, 8.3454e-01],
        [1.3843e-04, 1.9498e-01, 8.0488e-01],
        [9.9157e-01, 8.4282e-03, 1.9673e-09],
        [1.6890e-03, 4.4774e-01, 5.5057e-01],
        [9.9119e-01, 8.8128e-03, 4.2588e-09],
        [1.4590e-02, 9.4654e-01, 3.8873e-02],
        [2.8205e-03, 6.7344e-01, 3.2374e-01],
        [9.9193e-01, 8.0677e-03, 2.1019e-09],
        [1.1426e-03, 5.4821e-01, 4.5064e-01],
        [7.2897e-06, 3.3557e-02, 9.6644e-01],
        [3.4962e-03, 7.6010e-01, 2.3641e-01],
        [5.2541e-06, 2.6659e-02, 9.7334e-01],
        [3.1956e-05, 8.1120e-02, 9.1885e-01],
        [9.9731e-01, 2.6905e-03, 9.5620e-11],
        [2.8440e-03, 7.8686e-01, 2.1029e-01],
        [9.8745e-01, 1.2552e-02, 1.1724e-08],
        [2.2060e-05, 4.8153e-02, 9.5183e-01],
        [7.9103e-06, 2.0793e-02, 9.7920e-01],
        [9.2416e-03, 8.8334e-01, 1.0742e-01],
        [4.3166e-02, 9.2032e-01, 3

In [163]:
y_test

array([2., 2., 2., 2., 0., 1., 0., 1., 1., 0., 1., 2., 1., 2., 2., 0., 1.,
       0., 2., 2., 1., 1., 2., 2., 1., 0., 1., 1., 2., 2.])

In [156]:
#import pandas as pd
#from sklearn.metrics import r2_score, mean_squared_error

X = Variable(torch.tensor(x_train).float()) 
result = net(X)
pred=result.data[:,0].numpy()
pred
#print(len(pred),len(y_train))
#print (r2_score(pred,y_train))
#print (mean_squared_error(pred,y_train))
#print (np.mean(y_train))

array([4.7394770e-04, 4.0819177e-05, 2.2186257e-06, 1.3763828e-02,
       9.8616737e-01, 6.2114466e-04, 3.3937348e-03, 9.8896348e-01,
       9.7694379e-01, 2.4881689e-03, 3.4207554e-04, 9.9834800e-01,
       8.7975385e-03, 5.0197526e-05, 8.1698654e-06, 1.7256619e-06,
       9.9654359e-01, 9.9131060e-01, 2.3777692e-03, 9.8334634e-01,
       9.9392444e-01, 3.9395283e-04, 9.6809477e-01, 4.6868704e-04,
       9.9388039e-01, 9.8817885e-01, 9.9137479e-01, 2.9020704e-05,
       4.4036337e-06, 9.9477154e-01, 7.5791954e-06, 5.0870261e-05,
       9.7964913e-01, 9.9439114e-01, 1.1319418e-02, 3.9690685e-02,
       8.5031854e-05, 9.9303931e-01, 9.9749458e-01, 1.9643961e-02,
       1.3970531e-02, 9.9424779e-01, 3.1878965e-06, 8.5061016e-05,
       1.6658838e-05, 3.8102742e-06, 5.7934481e-06, 4.9523059e-03,
       9.9471259e-01, 9.8502004e-01, 4.2611419e-04, 9.7530431e-01,
       9.8616737e-01, 5.1592458e-03, 3.9662262e-03, 7.0456611e-03,
       6.1243493e-02, 1.6658838e-05, 1.3780198e-02, 3.7697493e

In [157]:
y_train

array([1., 2., 2., 1., 0., 2., 1., 0., 0., 1., 2., 0., 1., 2., 2., 2., 0.,
       0., 1., 0., 0., 2., 0., 2., 0., 0., 0., 2., 2., 0., 2., 2., 0., 0.,
       1., 1., 2., 0., 0., 1., 1., 0., 2., 2., 2., 2., 2., 1., 0., 0., 2.,
       0., 0., 1., 1., 1., 1., 2., 1., 2., 0., 2., 1., 0., 0., 2., 1., 2.,
       2., 0., 1., 1., 2., 0., 2., 1., 1., 0., 2., 2., 0., 0., 1., 1., 2.,
       0., 0., 1., 0., 1., 2., 0., 2., 0., 0., 1., 0., 0., 1., 2., 1., 1.,
       1., 0., 0., 1., 2., 0., 0., 1., 1., 1., 2., 1., 1., 1., 2., 0., 0.,
       1.])

In [132]:
torch.max(labels, 1)[1]

tensor([2, 0, 0, 1, 0, 2, 0, 1, 0, 1, 1, 1, 1, 1, 1, 2, 1, 0, 1, 1, 0, 1, 2, 0,
        0, 2, 1, 2, 2, 0, 0, 2, 0, 1, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1,
        1, 2, 1, 2, 1, 2, 1, 1, 2, 2, 0, 2, 1, 0, 2, 1, 0, 0, 0, 1, 0, 1, 2, 0,
        0, 2, 0, 2, 1, 0, 2, 2, 1, 2, 1, 0, 0, 0, 0, 0, 0, 2, 2, 0, 1, 1, 0, 2,
        0, 0, 1, 0, 1, 2, 0, 2, 1, 1, 2, 1, 0, 2, 0, 2, 2, 1, 2, 2, 1, 1, 2, 2])

In [133]:
net(inputs)

tensor([[1.7532e-05, 9.9998e-01, 9.9472e-01],
        [9.9841e-01, 3.2520e-03, 8.0627e-09],
        [9.9872e-01, 2.6295e-03, 6.5938e-09],
        [3.0683e-03, 9.9832e-01, 7.6827e-04],
        [9.9817e-01, 3.8096e-03, 8.4707e-09],
        [5.6918e-05, 9.9995e-01, 9.3180e-01],
        [9.9867e-01, 2.7122e-03, 6.9904e-09],
        [6.1177e-04, 9.9961e-01, 2.0079e-02],
        [9.9835e-01, 3.3730e-03, 8.2017e-09],
        [9.9123e-04, 9.9945e-01, 3.1019e-03],
        [9.0797e-03, 9.9558e-01, 1.1315e-04],
        [4.1772e-03, 9.9784e-01, 4.1455e-04],
        [2.2408e-03, 9.9869e-01, 2.0865e-03],
        [3.0383e-04, 9.9979e-01, 1.0099e-01],
        [2.1925e-05, 9.9998e-01, 9.9087e-01],
        [1.1378e-05, 9.9999e-01, 9.9884e-01],
        [2.4658e-03, 9.9867e-01, 1.1661e-03],
        [9.9862e-01, 2.8383e-03, 7.0753e-09],
        [5.8071e-03, 9.9707e-01, 1.8210e-04],
        [1.8411e-02, 9.9089e-01, 7.0277e-05],
        [9.9581e-01, 8.8533e-03, 1.4452e-08],
        [7.1969e-05, 9.9994e-01, 7